# Aplicando PyMFE ao conjunto de dados escolhidos

## Instalação de pacotes necessários

In [1]:
!pip install -Uq pymfe openml 

DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Imports

In [1]:
import os
import warnings

import openml
import pandas as pd
from pymfe.mfe import MFE
from tqdm.notebook import tqdm
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from config import dataset_ids

warnings.simplefilter(action='ignore', category=FutureWarning)

## Recuperando dados

In [2]:
datasets = openml.datasets.get_datasets(dataset_ids, download_data=True, download_qualities=True)

## Extraindo MetaFeatures dos Conjuntos de Dados

In [6]:
def get_mf(dataset: openml.datasets.dataset.OpenMLDataset) -> pd.DataFrame:
    X, y, categorical_indicator, _ = dataset.get_data(target=dataset.default_target_attribute)
    
    categorical_preprocessor = OneHotEncoder(handle_unknown='infrequent_if_exist',
                                             sparse_output=False )
    

    preprocessor = ColumnTransformer([
        ('one-hot-encoder', categorical_preprocessor, categorical_indicator)],
                                     remainder='passthrough' )
    
    X_t = preprocessor.fit_transform(X)
    
    extractor = MFE(groups='all')
    extractor.fit(X_t, y.to_numpy())
    
    results = extractor.extract(suppress_warnings=False)

    record = {k: v for k, v in zip(*results)}

    record['dataset_id'] = dataset.id

    return 

In [4]:
records = [get_mf(d) for d in tqdm(datasets)]

  0%|          | 0/70 [00:00<?, ?it/s]

/home/gabrielscn/.local/lib/python3.8/site-packages/pymfe/_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/home/gabrielscn/.local/lib/python3.8/site-packages/pymfe/_internal.py:731: RuntimeWarning: Can't summarize feature 'can_cor' with summary 'sd'. Will set it as 'np.nan'.
  warnings.warn(
/home/gabrielscn/.local/lib/python3.8/site-packages/pymfe/statistical.py:1267: RuntimeWarning: divide by zero encountered in log
  np.log(np.linalg.det(S_i)) for S_i in sample_cov_matrices
/home/gabrielscn/.local/lib/python3.8/site-packages/pymfe/statistical.py:1272: RuntimeWarning: divide by zero encountered in log
  * np.log(np.linalg.det(pooled_cov_mat))
/home/gabrielscn/.local/lib/python3.8/site-packages/pymfe/statistical.py:1271: RuntimeWarning: invalid value encountered in double_scalars
  (num_inst - num_classes)
/home/gabrielscn/.local/lib/python3.8/site-packages/pymfe/_internal.py:731: RuntimeWarni

Salvando resultados:

In [15]:
df = pd.DataFrame.from_records(records)

columns = list(df.columns)
df['dataset_id'] = dataset_ids
df = df[['dataset_id'] + columns]

df.to_csv('metafeatures.csv')